![iceberg-logo](https://www.apache.org/logos/res/iceberg/iceberg.png)

### [Docker, Spark, and Iceberg: The Fastest Way to Try Iceberg!](https://tabular.io/blog/docker-spark-and-iceberg/)

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

spark

25/06/06 10:34:58 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


## Load One Month of NYC Taxi/Limousine Trip Data

For this notebook, we will use the New York City Taxi and Limousine Commision Trip Record Data that's available on the AWS Open Data Registry. This contains data of trips taken by taxis and for-hire vehicles in New York City. We'll save this into an iceberg table called `taxis`.

To be able to rerun the notebook several times, let's drop the table if it exists to start fresh.

In [2]:
%%sql

DROP DATABASE IF EXISTS nyc

++
||
++
++

In [3]:
%%sql

SHOW DATABASES

namespace


In [4]:
%%sql

CREATE DATABASE IF NOT EXISTS nyc

++
||
++
++

In [5]:
%%sql

SHOW DATABASES

namespace
nyc


In [6]:
%%sql

DROP TABLE IF EXISTS nyc.taxis

++
||
++
++

In [7]:
%%sql

SHOW TABLES IN nyc

namespace,tableName,isTemporary


In [8]:
df = spark.read.parquet("/home/iceberg/data/yellow_tripdata_2021-04.parquet")
df.write.saveAsTable("nyc.taxis")

In [9]:
%%sql

SHOW TABLES IN nyc

namespace,tableName,isTemporary
nyc,taxis,False


In [10]:
%%sql

DESCRIBE EXTENDED nyc.taxis

col_name,data_type,comment
VendorID,bigint,None
tpep_pickup_datetime,timestamp_ntz,None
tpep_dropoff_datetime,timestamp_ntz,None
passenger_count,double,None
trip_distance,double,None
RatecodeID,double,None
store_and_fwd_flag,string,None
PULocationID,bigint,None
DOLocationID,bigint,None
payment_type,bigint,None


In [11]:
%%sql

SELECT COUNT(*) as cnt
FROM nyc.taxis

cnt
2171187


## Schema Evolution

Adding, dropping, renaming, or altering columns is easy and safe in Iceberg. In this example, we'll rename `fare_amount` to `fare` and `trip_distance` to `distance`. We'll also add a float column `fare_per_distance_unit` immediately after `distance`.

In [12]:
%%sql

ALTER TABLE nyc.taxis RENAME COLUMN fare_amount TO fare

++
||
++
++

In [13]:
%%sql

ALTER TABLE nyc.taxis RENAME COLUMN trip_distance TO distance

++
||
++
++

In [14]:
%%sql

ALTER TABLE nyc.taxis ALTER COLUMN distance COMMENT 'The elapsed trip distance in miles reported by the taximeter.'

++
||
++
++

In [15]:
%%sql

ALTER TABLE nyc.taxis ALTER COLUMN distance TYPE double;

++
||
++
++

In [16]:
%%sql

ALTER TABLE nyc.taxis ALTER COLUMN distance AFTER fare;

++
||
++
++

In [17]:
%%sql

ALTER TABLE nyc.taxis
ADD COLUMN fare_per_distance_unit float AFTER distance

++
||
++
++

Let's update the new `fare_per_distance_unit` to equal `fare` divided by `distance`.

In [18]:
%%sql

UPDATE nyc.taxis
SET fare_per_distance_unit = fare/distance

++
||
++
++

In [19]:
%%sql

DESCRIBE nyc.taxis

col_name,data_type,comment
VendorID,bigint,None
tpep_pickup_datetime,timestamp_ntz,None
tpep_dropoff_datetime,timestamp_ntz,None
passenger_count,double,None
RatecodeID,double,None
store_and_fwd_flag,string,None
PULocationID,bigint,None
DOLocationID,bigint,None
payment_type,bigint,None
fare,double,None


In [20]:
%%sql

SELECT
VendorID
,tpep_pickup_datetime
,tpep_dropoff_datetime
,fare
,distance
,fare_per_distance_unit
FROM nyc.taxis

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,fare,distance,fare_per_distance_unit
1,2021-04-01 00:00:18,2021-04-01 00:21:54,25.5,8.4,3.0357143878936768
1,2021-04-01 00:42:37,2021-04-01 00:46:23,5.0,0.9,5.55555534362793
1,2021-04-01 00:57:56,2021-04-01 01:08:22,11.5,3.4,3.382352828979492
1,2021-04-01 00:01:58,2021-04-01 00:54:27,44.2,0.0,None
2,2021-04-01 00:24:55,2021-04-01 00:34:33,9.0,1.96,4.591836929321289
2,2021-04-01 00:19:16,2021-04-01 00:21:46,4.5,0.77,5.844155788421631
2,2021-04-01 00:25:11,2021-04-01 00:31:53,11.5,3.65,3.1506848335266113
1,2021-04-01 00:27:53,2021-04-01 00:47:03,26.5,8.9,2.9775280952453613
2,2021-04-01 00:24:24,2021-04-01 00:37:50,12.0,2.98,4.026845455169678
1,2021-04-01 00:19:18,2021-04-01 00:41:25,28.0,8.9,3.1460673809051514


## Expressive SQL for Row Level Changes
With Iceberg tables, `DELETE` queries can be used to perform row-level deletes. This is as simple as providing the table name and a `WHERE` predicate. If the filter matches an entire partition of the table, Iceberg will intelligently perform a metadata-only operation where it simply deletes the metadata for that partition.

Let's perform a row-level delete for all rows that have a `fare_per_distance_unit` greater than 4 or a `distance` greater than 2. This should leave us with relatively short trips that have a relatively high fare per distance traveled.

In [21]:
%%sql

DELETE FROM nyc.taxis
WHERE fare_per_distance_unit > 4.0 OR distance > 2.0

++
||
++
++

There are some fares that have a `null` for `fare_per_distance_unit` due to the distance being `0`. Let's remove those as well.

In [22]:
%%sql

DELETE FROM nyc.taxis
WHERE fare_per_distance_unit is null

++
||
++
++

In [23]:
%%sql

SELECT
VendorID
,tpep_pickup_datetime
,tpep_dropoff_datetime
,fare
,distance
,fare_per_distance_unit
FROM nyc.taxis

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,fare,distance,fare_per_distance_unit
2,2021-04-01 00:53:54,2021-04-01 00:58:37,7.5,1.93,3.8860104084014893
2,2021-04-01 00:05:45,2021-04-01 00:15:21,-9.0,1.96,-4.591836929321289
2,2021-04-01 00:16:50,2021-04-01 00:22:58,7.5,1.95,3.846153736114502
2,2021-04-01 00:22:33,2021-04-01 00:23:00,-2.5,0.03,-83.33333587646484
1,2021-04-01 00:48:37,2021-04-01 00:54:30,7.5,1.9,3.9473683834075928
2,2021-04-01 00:26:08,2021-04-01 00:29:36,6.5,1.79,3.6312849521636963
2,2021-04-01 00:10:58,2021-04-01 00:16:00,6.5,1.66,3.9156627655029297
2,2021-04-01 00:13:14,2021-04-01 00:16:13,-4.5,0.73,-6.164383411407471
1,2021-04-01 00:20:52,2021-04-01 00:27:13,8.0,2.0,4.0
2,2021-04-01 01:12:20,2021-04-01 01:17:19,7.0,1.92,3.6458332538604736


In [24]:
%%sql

SELECT COUNT(*) as cnt
FROM nyc.taxis

cnt
17703


## Metadata Tables

Iceberg tables contain very rich metadata that can be easily queried. For example, you can retrieve the manifest list for any snapshot, simply by querying the table's `snapshots` table.

In [25]:
%%sql

SELECT snapshot_id, manifest_list
FROM nyc.taxis.snapshots

snapshot_id,manifest_list
6875512706475117218,s3://warehouse/nyc/taxis/metadata/snap-6875512706475117218-1-bf3d8d5c-d51e-4fe6-8e46-532f2af917bc.avro
7918471987921015577,s3://warehouse/nyc/taxis/metadata/snap-7918471987921015577-1-d6538511-eb22-4572-b128-3ce2e4c2e74b.avro
6871840922971322444,s3://warehouse/nyc/taxis/metadata/snap-6871840922971322444-1-be9a34f0-8165-47ce-b933-bf140a52fb69.avro
389044038313158510,s3://warehouse/nyc/taxis/metadata/snap-389044038313158510-1-dff44436-a9bf-497f-bb83-ef9e51bde3b3.avro


The `files` table contains loads of information on data files, including column level statistics such as null counts, lower bounds, and upper bounds.

In [26]:
%%sql

SELECT file_path, file_format, record_count, null_value_counts, lower_bounds, upper_bounds
FROM nyc.taxis.files

file_path,file_format,record_count,null_value_counts,lower_bounds,upper_bounds
s3://warehouse/nyc/taxis/data/00000-21-0d53c441-335a-4dfe-b89d-618771090ca9-0-00001.parquet,PARQUET,17703,"{1: 0, 2: 0, 3: 0, 4: 299, 5: 0, 6: 299, 7: 299, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 299, 19: 299, 20: 0}","{1: bytearray(b'\x01\x00\x00\x00\x00\x00\x00\x00'), 2: bytearray(b'@\x88-\xfe\xdd\xbe\x05\x00'), 3: bytearray(b'@\x98\x82 \xde\xbe\x05\x00'), 4: bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00'), 5: bytearray(b'{\x14\xaeG\xe1z\x84?'), 6: bytearray(b'\x00\x00\x00\x00\x00\x00\xf0?'), 7: bytearray(b'N'), 8: bytearray(b'\x01\x00\x00\x00\x00\x00\x00\x00'), 9: bytearray(b'\x01\x00\x00\x00\x00\x00\x00\x00'), 10: bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00'), 11: bytearray(b'\x00\x00\x00\x00\x00 l\xc0'), 12: bytearray(b'\x00\x00\x00\x00\x00\x00\x12\xc0'), 13: bytearray(b'\x00\x00\x00\x00\x00\x00\xe0\xbf'), 14: bytearray(b'\x85\xebQ\xb8\x1e\xd5t\xc0'), 15: bytearray(b'\x00\x00\x00\x00\x00\x00\x04\xc0'), 16: bytearray(b'333333\xd3\xbf'), 17: bytearray(b'R\xb8\x1e\x85\xebYu\xc0'), 18: bytearray(b'\x00\x00\x00\x00\x00\x00\x04\xc0'), 19: bytearray(b'\x00\x00\x00\x00\x00\x00\xf4\xbf'), 20: bytearray(b'\x00\x00\xfa\xc5')}","{1: bytearray(b'\x06\x00\x00\x00\x00\x00\x00\x00'), 2: bytearray(b'@\xa72h9\xc1\x05\x00'), 3: bytearray(b'\xc0X\xfclC\xc1\x05\x00'), 4: bytearray(b'\x00\x00\x00\x00\x00\x00\x18@'), 5: bytearray(b'\x00\x00\x00\x00\x00\x00\x00@'), 6: bytearray(b'\x00\x00\x00\x00\x00\xc0X@'), 7: bytearray(b'Y'), 8: bytearray(b'\t\x01\x00\x00\x00\x00\x00\x00'), 9: bytearray(b'\t\x01\x00\x00\x00\x00\x00\x00'), 10: bytearray(b'\x04\x00\x00\x00\x00\x00\x00\x00'), 11: bytearray(b'\x00\x00\x00\x00\x00\x00 @'), 12: bytearray(b'\x00\x00\x00\x00\x00\x00\x16@'), 13: bytearray(b'\x00\x00\x00\x00\x00\x00\xe0?'), 14: bytearray(b'\x00\x00\x00\x00\x00@U@'), 15: bytearray(b'\x00\x00\x00\x00\x00\x80;@'), 16: bytearray(b'333333\xd3?'), 17: bytearray(b'33333SU@'), 18: bytearray(b'\x00\x00\x00\x00\x00\x00\x04@'), 19: bytearray(b'\x00\x00\x00\x00\x00\x00\xf4?'), 20: bytearray(b'\x00\x00\x80@')}"


## Time Travel

The history table lists all snapshots and which parent snapshot they derive from. The `is_current_ancestor` flag let's you know if a snapshot is part of the linear history of the current snapshot of the table.

In [27]:
%%sql

SELECT *
FROM nyc.taxis.history

made_current_at,snapshot_id,parent_id,is_current_ancestor
2025-06-06 10:36:39.621000,6875512706475117218,None,True
2025-06-06 10:45:55.721000,7918471987921015577,6875512706475117218,True
2025-06-06 10:48:04.530000,6871840922971322444,7918471987921015577,True
2025-06-06 10:48:22.641000,389044038313158510,6871840922971322444,True


You can time-travel by altering the `current-snapshot-id` property of the table to reference any snapshot in the table's history. Let's revert the table to it's original state by traveling to the very first snapshot ID.

In [28]:
%%sql --var df

SELECT *
FROM nyc.taxis.history

made_current_at,snapshot_id,parent_id,is_current_ancestor
2025-06-06 10:36:39.621000,6875512706475117218,None,True
2025-06-06 10:45:55.721000,7918471987921015577,6875512706475117218,True
2025-06-06 10:48:04.530000,6871840922971322444,7918471987921015577,True
2025-06-06 10:48:22.641000,389044038313158510,6871840922971322444,True


In [31]:
original_snapshot = df.head().snapshot_id
spark.sql(f"CALL system.rollback_to_snapshot('nyc.taxis', {original_snapshot})")
original_snapshot

6875512706475117218

In [32]:
%%sql

SELECT
VendorID
,tpep_pickup_datetime
,tpep_dropoff_datetime
,fare
,distance
,fare_per_distance_unit
FROM nyc.taxis

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,fare,distance,fare_per_distance_unit
1,2021-04-01 00:00:18,2021-04-01 00:21:54,25.5,8.4,None
1,2021-04-01 00:42:37,2021-04-01 00:46:23,5.0,0.9,None
1,2021-04-01 00:57:56,2021-04-01 01:08:22,11.5,3.4,None
1,2021-04-01 00:01:58,2021-04-01 00:54:27,44.2,0.0,None
2,2021-04-01 00:24:55,2021-04-01 00:34:33,9.0,1.96,None
2,2021-04-01 00:19:16,2021-04-01 00:21:46,4.5,0.77,None
2,2021-04-01 00:25:11,2021-04-01 00:31:53,11.5,3.65,None
1,2021-04-01 00:27:53,2021-04-01 00:47:03,26.5,8.9,None
2,2021-04-01 00:24:24,2021-04-01 00:37:50,12.0,2.98,None
1,2021-04-01 00:19:18,2021-04-01 00:41:25,28.0,8.9,None


Another look at the history table shows that the original state of the table has been added as a new entry
with the original snapshot ID.

In [33]:
%%sql

SELECT *
FROM nyc.taxis.history

made_current_at,snapshot_id,parent_id,is_current_ancestor
2025-06-06 10:36:39.621000,6875512706475117218,None,True
2025-06-06 10:45:55.721000,7918471987921015577,6875512706475117218,False
2025-06-06 10:48:04.530000,6871840922971322444,7918471987921015577,False
2025-06-06 10:48:22.641000,389044038313158510,6871840922971322444,False
2025-06-06 10:53:31.073000,6875512706475117218,None,True


In [34]:
%%sql

SELECT COUNT(*) as cnt
FROM nyc.taxis

cnt
2171187


## Partitioning

A table’s partitioning can be updated in place and applied only to newly written data. Query plans are then split, using the old partition scheme for data written before the partition scheme was changed, and using the new partition scheme for data written after. People querying the table don’t even have to be aware of this split. Simple predicates in WHERE clauses are automatically converted to partition filters that prune out files with no matches. This is what’s referred to in Iceberg as *Hidden Partitioning*.

In [35]:
%%sql

DROP TABLE IF EXISTS nyc.taxis_partitioned

++
||
++
++

In [36]:
%%sql

SHOW TABLES IN nyc

namespace,tableName,isTemporary
nyc,taxis,False


In [37]:
df = spark.read.parquet("/home/iceberg/data/yellow_tripdata_2021-04.parquet")
df.write.saveAsTable("nyc.taxis_partitioned")

In [38]:
%%sql

ALTER TABLE nyc.taxis_partitioned ADD PARTITION FIELD days(tpep_pickup_datetime)

++
||
++
++

In [39]:
df = spark.read.parquet("/home/iceberg/data/yellow_tripdata_2021-05.parquet")
df.writeTo("nyc.taxis_partitioned").append()

In [40]:
df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [42]:
%%sql

DESCRIBE EXTENDED nyc.taxis_partitioned

col_name,data_type,comment
VendorID,bigint,None
tpep_pickup_datetime,timestamp_ntz,None
tpep_dropoff_datetime,timestamp_ntz,None
passenger_count,double,None
trip_distance,double,None
RatecodeID,double,None
store_and_fwd_flag,string,None
PULocationID,bigint,None
DOLocationID,bigint,None
payment_type,bigint,None


In [43]:
%%sql

SELECT *
FROM nyc.taxis_partitioned
WHERE days(tpep_pickup_datetime) = '2021-05-27'
LIMIT 1

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
2,2021-05-27 08:13:38,2021-05-27 08:55:40,1.0,15.82,1.0,N,138,87,1,47.5,0.0,0.5,11.72,6.55,0.3,70.32,2.5,1.25
